In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator='\n',
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

# cache_dir에 있는 embed를 확인하고 업으면 OpenAIEmbeddings 사용
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

# retriever : class의 interface. document를 retrieve
# from vector store, cloud 등
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions.")

/Users/dare/Desktop/bunkerkids/fullstack-gpt/venv/lib/python3.11/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'Victory Mansions is a building with glass doors and a hallway that smells like boiled cabbage and old rag mats. There is a large colored poster on one wall depicting the face of a man in his forties with a black mustache. The flat where Winston Smith lives is on the seventh floor, and the building has a malfunctioning lift. There is a telescreen on the right-hand wall that cannot be completely shut off, and it is currently reading out figures related to pig-iron production. The world outside the window appears cold, with little eddies of wind swirling dust and torn paper. The only color seems to come from the posters plastered everywhere. '